# Python 知识

# 代码设计

## 类继承关系
```mermaid
classDiagram
    %% 元类层次结构
    class Configured
    class DataUpdater
    class ScheduleManager
    class Data
    
    %% 继承关系
    Configured <|-- DataUpdater
    DataUpdater o-- ScheduleManager: has
    DataUpdater o-- Data: has
```

## class DataUpdater(Configured)
用于实现金融数据的定期更新和实时维护。包含：
- `self._data`：`Data` 实例，存储和管理实际的金融数据。
- `self._schedule_manager`: `ScheduleManager` 实例，负责任务调度和时间管理。

### `__init__`

```python
def __init__(self, data: Data, schedule_manager: tp.Optional[ScheduleManager] = None, **kwargs) -> None:
    Configured.__init__(
        self,
        data=data,
        schedule_manager=schedule_manager,
        **kwargs
    )
    self._data = data
    if schedule_manager is None:
        schedule_manager = ScheduleManager()
    self._schedule_manager = schedule_manager
```

### update
数据更新，子类可以重新定义。
- 调用数据 `self._data` 的 `update` 方法执行实际的数据更新。
- 更新 `Configured` 的配置。
- 获取更新后的时间索引 `self.data.wrapper.index`，并记录到日志系统 `logger`。

```python
def update(self, **kwargs) -> None:
    self._data = self.data.update(**kwargs)
    self.update_config(data=self.data)
    new_index = self.data.wrapper.index
    logger.info(f"Updated data has {len(new_index)} rows from {new_index[0]} to {new_index[-1]}")
```

### update_every
使用 `self._schedule_manager` 来执行定期数据更新。

参数；
- `*args`: 时间间隔参数，传递给 ScheduleManager.every()
- `to` (int, optional): 结束条件，指定最大执行次数
  - 例如：to=10 表示最多执行10次后自动停止
- `tags` (Iterable[Hashable], optional): 任务标签列表，用于标识和管理特定的更新任务组
  - 例如：tags=['stock_update', 'real_time']
- `in_background` (bool, optional): 是否在后台执行，默认False
    - True: 异步后台执行，不阻塞主线程
    - False: 同步前台执行，阻塞主线程直到完成
- `start_kwargs` (dict, optional): 传递给调度管理器start方法的参数
    常用参数：
    - max_workers: 最大工作线程数
    - timeout: 任务超时时间
- `**kwargs`: 传递给update方法的参数
    这些参数会在每次更新时传递给数据源的update方法

```python
def update_every(self, *args, to: int = None, tags: tp.Optional[tp.Iterable[tp.Hashable]] = None,
                    in_background: bool = False, start_kwargs: dict = None, **kwargs) -> None:
    if start_kwargs is None:
        start_kwargs = {}
    self.schedule_manager.every(*args, to=to, tags=tags).do(self.update, **kwargs)
    if in_background:
        self.schedule_manager.start_in_background(**start_kwargs)
    else:
        self.schedule_manager.start(**start_kwargs)
```

### 例子

```python
class MultiSourceUpdater(vbt.DataUpdater):
    def __init__(self, data_sources, **kwargs):
        # data_sources是包含多个Data实例的字典
        self.data_sources = data_sources
        super().__init__(list(data_sources.values())[0], **kwargs)
        
    def update(self, **kwargs):
        print("开始更新多个数据源...")
        
        for name, data_instance in self.data_sources.items():
            try:
                # 更新每个数据源
                updated_data = data_instance.update(**kwargs)
                print(f"✓ {name} 更新成功")
            except Exception as e:
                print(f"✗ {name} 更新失败: {e}")

# 创建多个数据源
data_sources = {
    'stocks': vbt.YFData.download(['AAPL', 'GOOGL', 'MSFT'], period='1d'),
    'crypto': vbt.BinanceData.download(['BTCUSDT', 'ETHUSDT'], 
                                        start='1 hour ago', interval='1m'),
    'synthetic': vbt.GBMData.download('TEST', start='1 hour ago', freq='1min')
}

# 创建多源更新器
multi_updater = MultiSourceUpdater(data_sources)

# 每5分钟更新所有数据源
multi_updater.update_every(5).minutes.tag('multi_update')
```

### 源码
```python
class DataUpdater(Configured):
    def __init__(self, data: Data, schedule_manager: tp.Optional[ScheduleManager] = None, **kwargs) -> None:
        Configured.__init__(
            self,
            data=data,
            schedule_manager=schedule_manager,
            **kwargs
        )
        self._data = data
        if schedule_manager is None:
            schedule_manager = ScheduleManager()
        self._schedule_manager = schedule_manager

    @property
    def data(self) -> Data:
        return self._data

    @property
    def schedule_manager(self) -> ScheduleManager:
        return self._schedule_manager

    def update(self, **kwargs) -> None:
        self._data = self.data.update(**kwargs)
        self.update_config(data=self.data)
        new_index = self.data.wrapper.index
        logger.info(f"Updated data has {len(new_index)} rows from {new_index[0]} to {new_index[-1]}")

    def update_every(self, *args, to: int = None, tags: tp.Optional[tp.Iterable[tp.Hashable]] = None,
                     in_background: bool = False, start_kwargs: dict = None, **kwargs) -> None:
        if start_kwargs is None:
            start_kwargs = {}
        self.schedule_manager.every(*args, to=to, tags=tags).do(self.update, **kwargs)
        if in_background:
            self.schedule_manager.start_in_background(**start_kwargs)
        else:
            self.schedule_manager.start(**start_kwargs)
```